In [1]:
alphabet1 = "abcdefghijklmnopqrstuvwxyz"
alphabet2 ="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
cpu_threads = 6

In [2]:
from KrypBz.proj5.zdroje.crypt import crypt
from KrypBz.tools.file_tools import read_file_lines
from KrypBz.proj5.zdroje.weak_passwords import weak_passwords
from multiprocessing import Pool
import threading
import numpy
import time

In [3]:
class Problem():
    def __init__(self, name, salt, fingerprint):
        self.name = name
        self.salt = salt
        self.fingerprint = fingerprint
        self.unlocked = False
        self.password = None

    def _check_password(self, password):
        encrypted = crypt(password, self.salt)
        if encrypted.decode() == self.fingerprint:
            self.password = password
            self.unlocked = True
        return self.unlocked

    def _try_vocabulary(self):
        # weak_passwords = []
        for p in weak_passwords:
            if self._check_password(p):
                return self.unlocked
            for i in range(len(p)): # check one Upper char variants
                if self._check_password(p[:i] + p[i].upper() + p[i+1:]):
                    return self.unlocked
        return self.unlocked

    def _try_passwords(self):
        for c1 in alphabet1:
            for c2 in alphabet1:
                for c3 in alphabet1:
                    for c4 in alphabet1:
                        for c5 in alphabet1:
                            for c6 in alphabet1:
                                if self._check_password(''.join([c1,c2,c3,c4,c5,c6])):
                                    return self.unlocked
                                for c7 in alphabet1:
                                    if self._check_password(''.join([c1,c2,c3,c4,c5,c6,c7])):
                                        return self.unlocked

        for c1 in alphabet2:
            for c2 in alphabet2:
                for c3 in alphabet2:
                    for c4 in alphabet2:
                        if self._check_password(''.join([c1,c2,c3,c4])):
                            return self.unlocked
                        for c5 in alphabet2:
                            if self._check_password(''.join([c1,c2,c3,c4,c5])):
                                    return self.unlocked
        return self.unlocked

    def attempt_unlocking(self):
        if self._try_vocabulary():
            return self.unlocked
        if self._try_passwords():
            return self.unlocked
        return self.unlocked

In [4]:
def unlock_users(users):
    for u in users:
        u.attempt_unlocking()

def unlock_users_parallel(users, thread_count):
    threads = []
    user_lists = numpy.array_split(users, thread_count)
    for i in range(thread_count):
        t = threading.Thread(target=unlock_users, args=([user_lists[i].tolist()]))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

In [5]:
from KrypBz.proj5.password_cracker_pool_function import unlock_users_pool

def unlock_users_parallel_pool(users, thread_count):
    with Pool(thread_count) as pool:
        return pool.map(unlock_users_pool, users)

In [6]:
def unlock_file(path):
    lines = read_file_lines(path)
    print(lines)
    print(len(lines))
    users = []
    for l in lines:
        users.append(Problem(*(l.rstrip().split(':'))))
    # unlock_users_parallel(users, cpu_threads)
    users = unlock_users_parallel_pool(users, cpu_threads)
    output = []
    for u in users:
        if u.unlocked:
            output.append(u)
    return output

def print_users(users):
    for u in users:
        print(u.name + ":\t" + u.password)

In [ ]:
if __name__ == "__main__":
    ts = time.time()
    unlocked = unlock_file("./zdroje/shadow1.txt")
    print_users(unlocked)
    print(time.time()-ts)

['styrak:XrMzPyGR:spohYVWZLKN70o2Z0FAYdg==\n', 'kanik:TBMT0KXT:iX/u1ErpVntILT4m9yCSrQ==\n', 'svrcek14:aDSlosC7:7jIy8VkUJNtm9QwW16x4qQ==\n', 'paluch:97sSrhFn:/eZB3PLEu8ZCYoHIViEc9Q==\n', 'ondrisek:5qNXOrEP:ytLbvN/gtRhK5QREvLNXaQ==\n', 'poliakov:FcISumF6:FAW0P9fXhx4QFl72ppukCA==\n', 'varackov:YKebnths:ZMLYzoNBPmAyzP4jRk/CgA==\n', 'pastorek6:nTrIezS3:lel033Nr50xOyM8oVGJ3xQ==\n', 'brezani7:BQwKPf1d:ZTSjn+WW419VVBhAbe4sfg==\n', 'stepanov:PIhXxYrO:6/sYBPSrOjWX6Iqv6ULUFA==\n', 'rychnavs:dydRYjXg:y1jrEADvFrceouCeGfLdLw==\n', 'glemba:eioGLHtt:oUYwPxFAnXhJzFDZFfM/fw==\n', 'kubik:SXlgkIDp:/Im4sWDmSnkMPLWNJNALnA==\n', 'fratrikova7:xt5nvQwh:3Owp+cBtZUDWQWWglG0QWA==\n', 'hajtinger:Md2Ol43H:xR/eVx0g1/4xw7Lqcs6mFA==\n', 'kochlan4:lGcH4jnN:Tr4eLNFL4hKTP9uiU6y3tw==\n', 'franek17:E4Hfvczh:rC2Vktgpyi8ZuA3xkWdbhQ==\n', 'balak:s1KvQ1My:Yw36hcWSfQ+7uOlE+6KKGg==\n', 'kopacova:FIPArVBO:CAM8IKapg2oqPhjjg3OAkw==\n', 'pallova:rBDtC0Sr:To7/6wbybvUFz8wyjpjeZQ==\n', 'urbanico:F91MuYfW:UvmTd4camIE7QH6Y2vijqw==\n', 'k

In [ ]:
if __name__ == "__main__":
    ts = time.time()
    unlocked = unlock_file("./zdroje/shadow2.txt")
    print_users(unlocked)
    print(time.time()-ts)

In [ ]:
if __name__ == "__main__":
    ts = time.time()
    unlocked = unlock_file("./zdroje/shadow3.txt")
    print_users(unlocked)
    print(time.time()-ts)

In [ ]:
if __name__ == "__main__":
    ts = time.time()
    unlocked = unlock_file("./zdroje/shadow4.txt")
    print_users(unlocked)
    print(time.time()-ts)